# 1 SETUP
***

In [4]:
# Notebook Config 
import sys
import os
proj_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(proj_root)


# Data Analysis
import pandas as pd
import numpy as np

# My Functions
from src.data_prep import DataPreparationPipeline


In [5]:
df_trans = pd.read_csv('../data/raw/train_transaction.csv')
df_id = pd.read_csv('../data/raw/train_identity.csv')

In [6]:
pipeline = DataPreparationPipeline(df_trans,df_id)
df_final = pipeline.feature_selection().df_merge()

# 2 DATA PREPARATION
***

## 2.2 EDA
***

Exploracao para entender quais tratamentos precisam ser realizados e em quais colunas.

In [7]:
df_final.describe().T

,count,mean,std,min,25%,50%,75%,max
TransactionID,590540.0,3.282270e+06,1.704744e+05,2987000.000,3.134635e+06,3.282270e+06,3.429904e+06,3.577539e+06
isFraud,590540.0,3.499001e-02,1.837546e-01,0.000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
TransactionDT,590540.0,7.372311e+06,4.617224e+06,86400.000,3.027058e+06,7.306528e+06,1.124662e+07,1.581113e+07
TransactionAmt,590540.0,1.350272e+02,2.391625e+02,0.251,4.332100e+01,6.876900e+01,1.250000e+02,3.193739e+04
card1,590540.0,9.898735e+03,4.901170e+03,1000.000,6.019000e+03,9.678000e+03,1.418400e+04,1.839600e+04
card2,581607.0,3.625555e+02,1.577932e+02,100.000,2.140000e+02,3.610000e+02,5.120000e+02,6.000000e+02
card3,588975.0,1.531949e+02,1.133644e+01,100.000,1.500000e+02,1.500000e+02,1.500000e+02,2.310000e+02
card5,586281.0,1.992789e+02,4.124445e+01,100.000,1.660000e+02,2.260000e+02,2.260000e+02,2.370000e+02
addr1,524834.0,2.907338e+02,1.017411e+02,100.000,2.040000e+02,2.990000e+02,3.300000e+02,5.400000e+02
addr2,524834.0,8.680063e+01,2.690623e+00,10.000,8.700000e+01,8.700000e+01,8.700000e+01,1.020000e+02


In [8]:

def df_info_completo(df):
    info_df = pd.DataFrame({
        "coluna": df.columns,
        "dtype": df.dtypes.values,
        "nulos": df.isnull().sum().values,
        "% nulos": (df.isnull().sum().values / len(df) * 100).round(2)
    })
    return info_df

info_df = df_info_completo(df_final)
display(info_df.head(50))
display(info_df.tail(27))

,coluna,dtype,nulos,% nulos
0,TransactionID,int64,0,0.00
1,isFraud,int64,0,0.00
2,TransactionDT,int64,0,0.00
3,TransactionAmt,float64,0,0.00
4,ProductCD,object,0,0.00
5,card1,int64,0,0.00
6,card2,float64,8933,1.51
7,card3,float64,1565,0.27
8,card4,object,1577,0.27
9,card5,float64,4259,0.72


,coluna,dtype,nulos,% nulos
50,V188,float64,450721,76.32
51,V194,float64,450721,76.32
52,V221,float64,449124,76.05
53,V242,float64,460110,77.91
54,V247,float64,460110,77.91
55,V250,float64,449124,76.05
56,id_02,float64,449668,76.15
57,id_05,float64,453675,76.82
58,id_06,float64,453675,76.82
59,id_13,float64,463220,78.44


Apesar de todas colunas presentes terem sido selecionadas criteriosamente, será avaliado apenas as que não estão anonimizada.

### VARIÁVEIS CATEGORICAS
***
Exploração das variáveis categóricas.

In [14]:
cat_cols = [
    c for c in info_df[info_df['dtype'] == 'object']['coluna'].to_list()
    if not c.startswith("id")
]

cat_cols = ["isFraud"] + cat_cols

df_cat_analysis = df_final[cat_cols].copy()
df_cat_analysis = df_cat_analysis.fillna('missing')
df_cat_analysis

,isFraud,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9,DeviceType,DeviceInfo
0,0,W,discover,credit,missing,missing,T,T,T,M2,F,T,missing,missing,missing,missing,missing
1,0,W,mastercard,credit,gmail.com,missing,missing,missing,missing,M0,T,T,missing,missing,missing,missing,missing
2,0,W,visa,debit,outlook.com,missing,T,T,T,M0,F,F,F,F,F,missing,missing
3,0,W,mastercard,debit,yahoo.com,missing,missing,missing,missing,M0,T,F,missing,missing,missing,missing,missing
4,0,H,mastercard,credit,gmail.com,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,mobile,SAMSUNG SM-G892A Build/NRD90M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,0,W,visa,debit,missing,missing,T,T,T,M0,T,F,F,F,T,missing,missing
590536,0,W,mastercard,debit,gmail.com,missing,T,F,F,M0,F,T,F,F,F,missing,missing
590537,0,W,mastercard,debit,gmail.com,missing,T,F,F,missing,missing,T,missing,missing,missing,missing,missing
590538,0,W,mastercard,debit,aol.com,missing,T,T,T,M0,F,T,missing,missing,missing,missing,missing


#### PRODUCTCD
***
Categoria do produto/serviço adquirido.

In [12]:
pd.crosstab(df_cat_analysis['ProductCD'],df_cat_analysis['isFraud'], normalize=True) * 100

isFraud,0,1
ProductCD,,
C,10.246723,1.356047
H,5.325634,0.266536
R,6.142344,0.241474
S,1.852880,0.116165
W,72.933417,1.518779


In [ ]:
pd.crosstab(df_cat_analysis['ProductCD'],df_cat_analysis['isFraud'], normalize="index") * 100

isFraud,0,1
ProductCD,,
C,88.312731,11.687269
H,95.233769,4.766231
R,96.217406,3.782594
S,94.100447,5.899553
W,97.960061,2.039939


A variável ProductCD apresenta diferenças claras na propensão à fraude. A categoria C apresenta a maior taxa de fraude (11,7%), enquanto W, responsável por mais de 70% das transações, apresenta apenas 2,1% de fraude. Isso indica que certas categorias carregam maior risco relativo, mesmo que sua frequência absoluta seja menor. Para análise e modelagem, a variável será transformada em uma feature contínua representando o risco de fraude associado a cada categoria.

#### CARD4 & CARD6
***
Atributos derivados de cartões de pagamento.

##### CARD4

In [15]:
pd.crosstab(df_cat_analysis['card4'],df_cat_analysis['isFraud'], normalize=True) * 100

isFraud,0,1
card4,,
american express,1.369763,0.040471
discover,1.039218,0.087039
mastercard,30.941342,1.100010
missing,0.260101,0.006943
visa,62.890575,2.264538


In [16]:
pd.crosstab(df_cat_analysis['card4'],df_cat_analysis['isFraud'], normalize='index') * 100

isFraud,0,1
card4,,
american express,97.130163,2.869837
discover,92.271839,7.728161
mastercard,96.566905,3.433095
missing,97.400127,2.599873
visa,96.524390,3.475610


A variável card4, que indica a bandeira do cartão, apresenta diferenças relevantes na propensão à fraude. A bandeira discover mostra a maior taxa de fraude (7,73%), enquanto american express e transações sem informação (missing) apresentam taxas mais baixas (2,6–2,87%). Visa e Mastercard apresentam taxas intermediárias (3,4%). Esses resultados sugerem que a bandeira do cartão é um indicador de risco relativo importante e, para modelagem, será transformada em uma feature contínua representando o risco de fraude por categoria
***

##### CARD6

In [17]:
pd.crosstab(df_cat_analysis['card6'],df_cat_analysis['isFraud'], normalize=True) * 100

isFraud,0,1
card6,,
charge card,0.002540,0.000000
credit,23.543875,1.684899
debit,72.690080,1.807498
debit or credit,0.005080,0.000000
missing,0.259424,0.006604


In [18]:
pd.crosstab(df_cat_analysis['card6'],df_cat_analysis['isFraud'], normalize='index') * 100

isFraud,0,1
card6,,
charge card,100.000000,0.000000
credit,93.321520,6.678480
debit,97.573749,2.426251
debit or credit,100.000000,0.000000
missing,97.517505,2.482495
